# Deep Convnets

In [1]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers
from keras.optimizers import RMSprop, Nadam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


### Load Data

In [2]:
data_path='../preprocessing/preprocessed_data'

In [ ]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)

In [4]:
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [3]:
training_generator = DataFeed.DataGenerator(data_path, ['train/voxforge', 'train/youtube'], num=-1, batch_size=64)

In [5]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=5),
             keras.callbacks.ModelCheckpoint('conv2d_deep.h5', monitor='val_loss', save_best_only=True)]

## Topcoder_net
based on [repo](https://github.com/pietz/language-recognition/blob/master/Language%20Classifier.ipynb)

In [24]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=192, n_hop=418,
                         fmin=0.0, fmax=8000, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False))
model.add(Normalization2d(str_axis='data_sample'))
model.add(layers.Conv2D(16, (3, 3), activation='elu', padding='same'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(32, (3, 3), activation='elu', padding='same'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(64, (3, 3), activation='elu', padding='same'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(128, (3, 3), activation='elu', padding='same'))
model.add(layers.MaxPooling2D())

model.add(layers.Conv2D(256, (3, 3), activation='elu', padding='same'))
model.add(layers.MaxPooling2D())

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

=> up to 86% val acc

## Resnet

In [14]:
from keras.applications import vgg16, resnet50
from keras_contrib.applications.resnet import ResNet18, ResNet

In [41]:
datagen = ImageDataGenerator(
                            featurewise_center=True,
                            featurewise_std_normalization=True,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            dtype='float32')

In [43]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=192, n_hop=418,
                         fmin=0.0, fmax=8000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
#model.add(layers.Lambda(baum)) try using data augmentation
model.add(ResNet(input_shape=(192, 192, 1), classes=3, block='basic', repetitions=[2, 2, 2], dropout=0.5))
model.summary()

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_18 (Melspectr (None, 192, 192, 1)       312512    
_________________________________________________________________
model_8 (Model)              (None, 3)                 2788419   
Total params: 3,100,931
Trainable params: 2,783,811
Non-trainable params: 317,120
_________________________________________________________________


=> up to 76% val_acc, overfits

## Mobilenet

In [6]:
from keras.applications.mobilenetv2 import MobileNetV2

In [7]:
input_tensor = layers.Input(shape=(1, 80000))
x = Melspectrogram(n_dft=512,
                   padding='same', sr=16000, n_mels=223, n_hop=360,
                   fmin=0.0, fmax=10000, power_melgram=1.0,
                   return_decibel_melgram=False, trainable_fb=False,
                   trainable_kernel=False)(input_tensor)

inception_model = MobileNetV2(include_top=True, input_tensor=x, weights=None,
                              alpha=0.25, classes=3)

x = inception_model.output


model = models.Model(input_tensor, outputs=x)
model.summary()

/home/joel/.local/share/virtualenvs/deepLID-B2i2hIZa/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 80000)     0                                            
__________________________________________________________________________________________________
melspectrogram_1 (Melspectrogra (None, 223, 223, 1)  320479      input_1[0][0]                    
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 1)  0           melspectrogram_1[0][0]           
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 8)  72          Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (

### Train Model

In [8]:
model.compile(optimizer=RMSprop(),
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [9]:
history = model.fit_generator(generator=training_generator,
                        epochs=10,
                        validation_data=(val_data, val_labels), 
                        shuffle=True,
                        use_multiprocessing=True,
                        workers=8,
                        max_queue_size=20,          
                        callbacks=callbacks)

Epoch 1/10
1250/1250 [==============================] - 224s 179ms/step - loss: 0.8516 - acc: 0.6062 - val_loss: 1.8429 - val_acc: 0.4840
Epoch 2/10
1250/1250 [==============================] - 237s 190ms/step - loss: 0.5446 - acc: 0.7758 - val_loss: 1.5758 - val_acc: 0.5492
Epoch 3/10
1250/1250 [==============================] - 236s 189ms/step - loss: 0.4146 - acc: 0.8345 - val_loss: 1.8104 - val_acc: 0.6159
Epoch 4/10
1250/1250 [==============================] - 237s 189ms/step - loss: 0.3398 - acc: 0.8676 - val_loss: 2.0582 - val_acc: 0.5786
Epoch 5/10
1250/1250 [==============================] - 237s 190ms/step - loss: 0.2943 - acc: 0.8873 - val_loss: 1.3236 - val_acc: 0.7129
Epoch 6/10
1250/1250 [==============================] - 238s 190ms/step - loss: 0.2569 - acc: 0.9022 - val_loss: 1.4442 - val_acc: 0.6843
Epoch 7/10
1250/1250 [==============================] - 239s 191ms/step - loss: 0.2321 - acc: 0.9120 - val_loss: 0.8766 - val_acc: 0.7900
Epoch 8/10
1250/1250 [============

In [10]:
model.compile(optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [11]:
history2 = model.fit_generator(generator=training_generator,
                        epochs=5,
                        validation_data=(val_data, val_labels), 
                        shuffle=True,
                        use_multiprocessing=True,
                        workers=8,
                        max_queue_size=20,          
                        callbacks=callbacks)

Epoch 1/5
1250/1250 [==============================] - 224s 179ms/step - loss: 0.1675 - acc: 0.9376 - val_loss: 1.0360 - val_acc: 0.7692
Epoch 2/5
1250/1250 [==============================] - 222s 178ms/step - loss: 0.1528 - acc: 0.9429 - val_loss: 0.9764 - val_acc: 0.7794
Epoch 3/5
1250/1250 [==============================] - 221s 177ms/step - loss: 0.1447 - acc: 0.9454 - val_loss: 0.9363 - val_acc: 0.7946
Epoch 4/5
1250/1250 [==============================] - 223s 179ms/step - loss: 0.1358 - acc: 0.9489 - val_loss: 1.0418 - val_acc: 0.7647
Epoch 5/5
1250/1250 [==============================] - 227s 181ms/step - loss: 0.1262 - acc: 0.9528 - val_loss: 0.7588 - val_acc: 0.8317
